<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [1]:
# import cell
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split
import sklearn.linear_model
import matplotlib.pyplot as plt
import tempfile
from sklearn.utils import shuffle
import os
import datetime
# Our Model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorboard.plugins.hparams import api as hp

In [8]:
# Your Code Starts Here - load in data and spilt
data = np.load('quickdraw10.npz')
def load_quickdraw10npz(path):
    X = path['arr_0']
    y = path['arr_1']
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9, test_size = 0.1)
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = load_quickdraw10npz(path=data)
len(X_train), len(X_test), len(y_train), len(y_test)

(90000, 10000, 90000, 10000)

In [9]:
X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = y_train / 255.0
y_test = y_test / 255.0
print(f'shapes{X_train.shape,y_train.shape,X_test.shape,y_test.shape}')

shapes((90000, 784), (90000,), (10000, 784), (10000,))


In [35]:
def model(opt, drop, activation, input_shape, output_size, n_layers):
    opt = opt(lr=.001)
    model = Sequential()
    model.add(Dense(50, activation=activation, input_shape=input_shape))
    model.add(Dropout(drop))
    for _ in range(n_layers):
        model.add(Dense(50, activation=activation))
        model.add(Dropout(drop))
    if output_size == 1:
        model.add(Dense(output_size, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', metrics=['mae', 'mse'], optimizer='adam')
    else:
        model.add(Dense(output_size, activation='softmax'))
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
    return model""""""

In [37]:
# create model
model = KerasClassifier(build_fn=model(SGD,drop=.02,activation='softmax',input_shape=(32,0),output_size=526,n_layers=2), verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_sizeiii, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [32,100,512],
              'epochs': [25],
              'units': [32],
              # TODO add more params
              }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

NameError: name 'Dropout' is not defined

In [4]:
# Function to create model, required for KerasClassifier
def create_model(units=32):
    opt = tf.keras.optimizers.Adagrad(
    learning_rate=0.001, initial_accumulator_value=0.1, epsilon=1e-07,
    name='Adagrad')
    model = Sequential(
      [
       Dense(50, activation='relu', input_dim=784),
       Dense(50, activation='relu')
      ])
    model.add(Dense(units, input_dim=784, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_sizeiii, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [32,64,512],
              'epochs': [20],
              'units': [32],
              # TODO add more params
              }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

ValueError: Unknown activation function: iisoftmax

In [147]:
from keras.layers import Dropout
for hl1 in [250, 200, 150, 100, 75, 50, 25, 15, 10, 7]:
    def baseline_model(units=32):
        opt = tf.keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax")
        model = Sequential()
        model.add(Dense(hl1, input_dim=784, kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.2,noise_shape=None, seed=None))
        model.add(Dense(100, kernel_initializer='normal', activation='sigmoid'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model
# create model
model = KerasClassifier(build_fn=baseline_model, verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10,32,64,512],
              'epochs': [20,50],
              'units': [32],
              # TODO add more params
              }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test),verbose=2)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
1094/1094 - 1s - loss: 2.4421 - accuracy: 0.1056 - val_loss: 2.1326 - val_accuracy: 0.0000e+00
Epoch 2/50
1094/1094 - 1s - loss: 1.8255 - accuracy: 0.3033 - val_loss: 4.6850 - val_accuracy: 0.0891
Epoch 3/50
1094/1094 - 1s - loss: 1.5536 - accuracy: 0.4417 - val_loss: 5.5766 - val_accuracy: 0.0909
Epoch 4/50
1094/1094 - 1s - loss: 1.4474 - accuracy: 0.4951 - val_loss: 5.6107 - val_accuracy: 0.0925
Epoch 5/50
1094/1094 - 1s - loss: 1.3870 - accuracy: 0.5235 - val_loss: 5.8763 - val_accuracy: 0.0917
Epoch 6/50
1094/1094 - 1s - loss: 1.3479 - accuracy: 0.5435 - val_loss: 6.0069 - val_accuracy: 0.0915
Epoch 7/50
1094/1094 - 1s - loss: 1.3146 - accuracy: 0.5554 - val_loss: 6.2407 - val_accuracy: 0.0913
Epoch 8/50
1094/1094 - 1s - loss: 1.2943 - accuracy: 0.5645 - val_loss: 6.2886 - val_accuracy: 0.0909
Epoch 9/50
1094/1094 - 1s - loss: 1.2785 - accuracy: 0.5694 - val_loss: 6.1062 - val_accuracy: 0.0924
Epoch 10/50
1094/1094 - 1s - loss: 1.2666 - accuracy: 0.5746 - val_loss: 6.171

In [171]:
def create_model2(lr,opt):
    """Insert Doc String"""
    opt = opt(lr)
    model = Sequential([
        Dense(32, activation="relu", input_dim=784),
        Dense(32, activation="relu"),
        Dense(10, activation="softmax")
      ])
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
def create_model3(lr,opt):
    """Insert Doc String"""
    opt = opt(lr)
    model = Sequential([
        Dense(32, activation="relu", input_dim=784),
        Dense(32, activation="relu"),
        Dense(10, activation="softmax")
      ])
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
# create model
model = KerasClassifier(build_fn=create_model3(.001, SGD))

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [100,512],
              'epochs': [20],
              'units': [32],
              # TODO add more params
              }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

TypeError: can't pickle _thread.RLock objects

In [97]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### 1. Create Experiment Configuration

In [98]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16,32]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,.01))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adagrad']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
      hparams=[HP_NUM_UNITS, HP_LEARNING_RATE, HP_OPTIMIZER],
      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
  )

### 2. Adapt Model Function with HParams

In [99]:
def train_test_model(hparams):
  
  model = tf.keras.Sequential(
      [tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
       tf.keras.layers.Dense(10, activation='softmax')      
  ])

  opt_name = hparams[HP_OPTIMIZER]
  lr = hparams[HP_LEARNING_RATE]

  if opt_name == 'adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
  elif opt_name == 'Adagrad':
    opt = tf.keras.optimizers.Adagrad(learning_rate=lr, initial_accumulator_value=0.1, epsilon=1e-07)
  else:
    raise ValueError(f'Unexpected optimizer: {opt_name}')

  model.compile(
      optimizer=opt,
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )

  model.fit(X_train, y_train, epochs=5)
  _, accuracy = model.evaluate(X_test, y_test)

  # Python convention: if a variable doesn't need a name, give it _
  # ten_ones = [1 for _ in range(10)]

  return accuracy

In [100]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [101]:
session_num = 0

# Basically a grid search
for num_units in HP_NUM_UNITS.domain.values:
  for learning_rate in (HP_LEARNING_RATE.domain.min_value,
                        HP_LEARNING_RATE.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_LEARNING_RATE: learning_rate,
          HP_OPTIMIZER: optimizer
      }

      run_name = f'run-{session_num}'
      print(f'--- Starting trial: {run_name}')
      print({param.name: hparams[param] for param in hparams})
      run('logs/hparams_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'learning_rate': 0.001, 'optimizer': 'Adagrad'}
Epoch 1/5
2188/2188 [==============================] - ETA: 0s - loss: 2.2148 - accuracy: 0.06 - ETA: 1s - loss: 1.2655 - accuracy: 0.08 - ETA: 1s - loss: 0.9208 - accuracy: 0.09 - ETA: 1s - loss: 0.7309 - accuracy: 0.09 - ETA: 1s - loss: 0.6096 - accuracy: 0.09 - ETA: 1s - loss: 0.5236 - accuracy: 0.09 - ETA: 1s - loss: 0.4626 - accuracy: 0.09 - ETA: 1s - loss: 0.4151 - accuracy: 0.10 - ETA: 0s - loss: 0.3775 - accuracy: 0.10 - ETA: 0s - loss: 0.3461 - accuracy: 0.10 - ETA: 0s - loss: 0.3200 - accuracy: 0.09 - ETA: 0s - loss: 0.2998 - accuracy: 0.09 - ETA: 0s - loss: 0.2798 - accuracy: 0.09 - ETA: 0s - loss: 0.2641 - accuracy: 0.09 - ETA: 0s - loss: 0.2496 - accuracy: 0.10 - ETA: 0s - loss: 0.2364 - accuracy: 0.10 - ETA: 0s - loss: 0.2248 - accuracy: 0.10 - ETA: 0s - loss: 0.2140 - accuracy: 0.10 - ETA: 0s - loss: 0.2043 - accuracy: 0.09 - ETA: 0s - loss: 0.1959 - accuracy: 0.09 - ETA: 0s - los

### 3. Visualize the Result

In [70]:
%tensorboard --logdir logs/hparams_tuning

Reusing TensorBoard on port 6006 (pid 13036), started 0:21:51 ago. (Use '!kill 13036' to kill it.)

### 4. Hyperparameters with RandomSearchCV (Learn)

In [103]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

"""
This model Tunes:
- Number of Neurons in the Hidden Layer
- Learning Rate in Adam

"""
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(Dense(32, input_shape=(16,), kernel_initializer = 'he_uniform', kernel_regularizer = None, kernel_constraint = 'MaxNorm', activation = 'relu')) 
    model.add(Dense(50, activation = 'relu')) 
    model.add(Dense(8))
    model.add(Dense(50))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adagrad(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [105]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='keras-tuner-trial4',
    project_name='helloworld')

In [106]:
tuner.search_space_summary()

In [107]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Epoch 1/5
2188/2188 [==============================] - ETA: 0s - loss: 2.3287 - accuracy: 0.0000e+ - ETA: 4s - loss: 2.2166 - accuracy: 0.0247   - ETA: 4s - loss: 2.0815 - accuracy: 0.05 - ETA: 4s - loss: 1.8900 - accuracy: 0.07 - ETA: 4s - loss: 1.6451 - accuracy: 0.08 - ETA: 4s - loss: 1.4238 - accuracy: 0.08 - ETA: 4s - loss: 1.2396 - accuracy: 0.08 - ETA: 4s - loss: 1.0744 - accuracy: 0.08 - ETA: 4s - loss: 0.9543 - accuracy: 0.08 - ETA: 4s - loss: 0.8654 - accuracy: 0.08 - ETA: 4s - loss: 0.7883 - accuracy: 0.08 - ETA: 4s - loss: 0.7290 - accuracy: 0.08 - ETA: 4s - loss: 0.6712 - accuracy: 0.09 - ETA: 4s - loss: 0.6199 - accuracy: 0.09 - ETA: 4s - loss: 0.5776 - accuracy: 0.09 - ETA: 4s - loss: 0.5390 - accuracy: 0.09 - ETA: 4s - loss: 0.5067 - accuracy: 0.09 - ETA: 3s - loss: 0.4780 - accuracy: 0.09 - ETA: 3s - loss: 0.4514 - accuracy: 0.09 - ETA: 3s - loss: 0.4286 - accuracy: 0.09 - ETA: 3s - loss: 0.4071 - accuracy: 0.09 - ETA: 3s - loss: 0.3885 - accuracy: 0.09 - ETA: 3s - los

Epoch 1/5
2188/2188 [==============================] - ETA: 0s - loss: 2.4183 - accuracy: 0.0000e+ - ETA: 2s - loss: 2.3662 - accuracy: 0.0000e+ - ETA: 2s - loss: 2.3410 - accuracy: 0.0000e+ - ETA: 2s - loss: 2.3194 - accuracy: 0.0000e+ - ETA: 2s - loss: 2.2999 - accuracy: 1.8275e- - ETA: 2s - loss: 2.2815 - accuracy: 4.3808e- - ETA: 2s - loss: 2.2632 - accuracy: 6.0798e- - ETA: 2s - loss: 2.2448 - accuracy: 7.2674e- - ETA: 2s - loss: 2.2267 - accuracy: 0.0014   - ETA: 2s - loss: 2.2103 - accuracy: 0.00 - ETA: 2s - loss: 2.1940 - accuracy: 0.00 - ETA: 2s - loss: 2.1783 - accuracy: 0.00 - ETA: 1s - loss: 2.1597 - accuracy: 0.00 - ETA: 1s - loss: 2.1425 - accuracy: 0.00 - ETA: 1s - loss: 2.1251 - accuracy: 0.00 - ETA: 1s - loss: 2.1073 - accuracy: 0.00 - ETA: 1s - loss: 2.0902 - accuracy: 0.01 - ETA: 1s - loss: 2.0742 - accuracy: 0.01 - ETA: 1s - loss: 2.0569 - accuracy: 0.01 - ETA: 1s - loss: 2.0385 - accuracy: 0.01 - ETA: 1s - loss: 2.0198 - accuracy: 0.01 - ETA: 1s - loss: 2.0019 - ac

Epoch 1/5
2188/2188 [==============================] - ETA: 0s - loss: 2.2607 - accuracy: 0.0000e+ - ETA: 3s - loss: 2.2355 - accuracy: 0.0000e+ - ETA: 3s - loss: 2.2213 - accuracy: 0.0000e+ - ETA: 3s - loss: 2.2092 - accuracy: 0.0000e+ - ETA: 3s - loss: 2.1955 - accuracy: 0.0000e+ - ETA: 3s - loss: 2.1839 - accuracy: 0.0000e+ - ETA: 3s - loss: 2.1709 - accuracy: 1.9172e- - ETA: 3s - loss: 2.1596 - accuracy: 3.2895e- - ETA: 3s - loss: 2.1486 - accuracy: 7.2005e- - ETA: 3s - loss: 2.1376 - accuracy: 0.0010   - ETA: 3s - loss: 2.1270 - accuracy: 0.00 - ETA: 3s - loss: 2.1166 - accuracy: 0.00 - ETA: 3s - loss: 2.1056 - accuracy: 0.00 - ETA: 3s - loss: 2.0947 - accuracy: 0.00 - ETA: 3s - loss: 2.0841 - accuracy: 0.00 - ETA: 3s - loss: 2.0737 - accuracy: 0.00 - ETA: 3s - loss: 2.0633 - accuracy: 0.00 - ETA: 3s - loss: 2.0533 - accuracy: 0.00 - ETA: 3s - loss: 2.0427 - accuracy: 0.01 - ETA: 3s - loss: 2.0319 - accuracy: 0.01 - ETA: 3s - loss: 2.0214 - accuracy: 0.01 - ETA: 3s - loss: 2.0109 

KeyboardInterrupt: 

In [ ]:
tuner.results_summary()

### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 